In [1]:
import pandas as pd
import bundestag_api
import re
import datetime
# you might need to install lxml
from src.load_data_website import get_df
from src.load_data import get_metadata, get_textdata
from src.people import *
from src.preprocessing import *

In [45]:
#################################################
### FIRST TRY: GET DATA FROM DOWNLOAD FOLDER ####
##################################################
# Try with downloaded data from website
df = get_df()
#mytext = str(df['TEXT'][0])
#df.to_csv('data/protokolle.csv')

# To read the full text in the TEXT column, we have to change max column width, ANY TIME, we want access to the text!
with pd.option_context('display.max_colwidth', None):
    # display the dataframe
    display(df.iloc[0:1,5:6])

In [6]:
###############################################
### USE API TO HAVE A LOOK AT THE PROTOCOLS ###
###############################################

# Look into specific protocol
bta = bundestag_api.btaConnection(apikey = 'rgsaY4U.oZRQKUHdJhF9qguHMkwCGIoLaqEcaHjYLF')
myprotocol = bta.get_plenaryprotocol('613')
text_file = open(r'text.txt', 'w')
text_file.write(myprotocol['text'])
text_file.close()


In [7]:
# Look into specific protocol -> pdf
bta = bundestag_api.btaConnection(apikey = 'rgsaY4U.oZRQKUHdJhF9qguHMkwCGIoLaqEcaHjYLF')
bta.get_plenaryprotocol('613')['fundstelle']
# oder direkt über Anpassung des Links mit Sitzungsnummer: https://dserver.bundestag.de/btp/18/18001.pdf

{'pdf_url': 'https://dserver.bundestag.de/btp/18/18001.pdf',
 'id': '613',
 'dokumentnummer': '18/1',
 'datum': '2013-10-22',
 'xml_url': 'https://dserver.bundestag.de/btp/18/18001.xml',
 'dokumentart': 'Plenarprotokoll',
 'herausgeber': 'BT',
 'urheber': []}

In [2]:
######################################################
### WE USED THIS SOLUTION INSTEAD TO READ THE DATA ###
######################################################

# Try API package instead (takes ~ 2 min.)
metadata = get_metadata()
textdata = get_textdata()
textdata

Plenarprotkolle (Bundestag) aus den Wahlperioden 18 und 19 geladen. Anzahl: 484
Plenarprotkolle (Bundestag) aus den Wahlperioden 18 und 19 geladen. Anzahl: 484


,id,text
0,5428,Präsident Dr. Wolfgang Schäuble:\nGuten Morgen...
1,5427,"Präsident Dr. Wolfgang Schäuble:\nGuten Tag, l..."
2,5425,Präsident Dr. Wolfgang Schäuble:\nGuten Morgen...
3,5424,Präsident Dr. Wolfgang Schäuble:\nGuten Morgen...
4,5423,Präsident Dr. Wolfgang Schäuble:\nLiebe Kolleg...
...,...,...
479,620,Präsident Dr. Norbert Lammert: \nDie Sitzung i...
480,619,Präsident Dr. Norbert Lammert: \nDie Sitzung i...
481,618,Präsident Dr. Norbert Lammert: \nDie Sitzung i...
482,615,Präsident Dr. Norbert Lammert: \nDie Sitzung i...


In [17]:
# Takes ~ 1 min. Results in a data frame with column main_text, which is the relevant part
df = clean_and_split_text(textdata)

# I recommend saving it in a csv, saves time for later
#df.to_csv('df.csv')
#df = pd.read_csv('df.csv')
df

,Unnamed: 0,id,text,main_text,interruptions,neutral_text
0,0,5428,Präsident Dr. Wolfgang Schäuble:\nGuten Morgen...,Britta Haßelmann (BÜNDNIS 90/DIE GRÜNEN):\nSeh...,"['(Beifall)', '(Beifall bei Abgeordneten der A...","['Präsident Dr. Wolfgang Schäuble:', 'Guten Mo..."
1,1,5427,"Präsident Dr. Wolfgang Schäuble:\nGuten Tag, l...","Interfraktionell ist vereinbart worden, die no...","['(Beifall bei der CDU/CSU, der SPD, der FDP u...","['Präsident Dr. Wolfgang Schäuble:', 'Guten Ta..."
2,2,5425,Präsident Dr. Wolfgang Schäuble:\nGuten Morgen...,Damit rufe ich den Tagesordnungspunkt 34 auf:\...,['(Beifall bei der SPD sowie bei Abgeordneten ...,"['Präsident Dr. Wolfgang Schäuble:', 'Guten Mo..."
3,3,5424,Präsident Dr. Wolfgang Schäuble:\nGuten Morgen...,Dann rufe ich den Tagesordnungspunkt 8 auf:\nA...,"['(Beifall)', '(Beifall bei der CDU/CSU sowie ...","['Präsident Dr. Wolfgang Schäuble:', 'Guten Mo..."
4,4,5423,Präsident Dr. Wolfgang Schäuble:\nLiebe Kolleg...,"Interfraktionell ist vereinbart worden, die no...","['(11. Ausschuss)', '(11. Ausschuss)', '(Ergän...","['Präsident Dr. Wolfgang Schäuble:', 'Liebe Ko..."
...,...,...,...,...,...,...
479,479,620,Präsident Dr. Norbert Lammert: \nDie Sitzung i...,"Guten Morgen, liebe Kolleginnen und Kollegen! ...","['(Beifall bei der CDU/CSU und der SPD)', '(Be...","['Präsident Dr. Norbert Lammert:', 'Die Sitzun..."
480,480,619,Präsident Dr. Norbert Lammert: \nDie Sitzung i...,Wir kommen nun zu unserem Tagesordnungspunkt 1...,"['(Die Anwesenden erheben sich)', '(Beifall be...","['Präsident Dr. Norbert Lammert:', 'Die Sitzun..."
481,481,618,Präsident Dr. Norbert Lammert: \nDie Sitzung i...,Wir kommen damit zu Tagesordnungspunkt 1 mit d...,"['(Beifall bei der CDU/CSU)', '(CDU/CSU)', '(H...","['Präsident Dr. Norbert Lammert:', 'Die Sitzun..."
482,482,615,Präsident Dr. Norbert Lammert: \nDie Sitzung i...,Dr. Gregor Gysi (DIE LINKE):\nHerr Präsident! ...,"['(Beifall)', '(DIE LINKE)', '(Beifall)', '(He...","['Präsident Dr. Norbert Lammert:', 'Die Sitzun..."


In [18]:
# Assign text to parties
df_text_of_parties = party_shares(df)
df_text_of_parties

,id,party,text,text_type
0,5428,BÜNDNIS 90/DIE GRÜNEN,Sehr geehrter Herr Präsident! Meine Damen und ...,main_text
1,5428,FDP,Herr Präsident! Meine Damen und Herren! Der An...,main_text
2,5428,CDU/CSU,Sehr geehrter Herr Präsident! Liebe Kolleginne...,main_text
3,5428,AfD,Herr Präsident! Meine Damen und Herren! Der Ei...,main_text
4,5428,SPD,Herr Präsident! Meine sehr verehrten Damen und...,main_text
...,...,...,...,...
4763,615,DIE LINKE,Na los!\nJa!\nDas stimmt nicht!\nKönnen!\nAnge...,interruption
4764,615,SPD,Er hat aber von Ungarn gesprochen!\nWir auch!\...,interruption
4765,613,CDU/CSU,Hier im Haus!\nStimmt!\nHeiterkeit\nNein!\nDoc...,interruption
4766,613,SPD,Hinter Ihnen!\nZuruf:\nNein!\nZuruf:\nEiner ko...,interruption


In [31]:
# Adding Metadata, takes ~30 seconds
df_text_of_parties = add_metadata(df_text_of_parties)

# Clean data
party_filter = ['SPD', 'CDU/CSU', 'DIE LINKE', 'BÜNDNIS 90/DIE GRÜNEN', 'AfD', 'FDP']
df_text_of_parties = df_text_of_parties[df_text_of_parties['party'].isin(party_filter)]

#df_text_of_parties.to_csv("df_party_shares.csv")
df_text_of_parties

Plenarprotkolle (Bundestag) aus den Wahlperioden 18 und 19 geladen. Anzahl: 484


,id,party,datum,wahlperiode
0,5428,BÜNDNIS 90/DIE GRÜNEN,2021-09-07,19.0
1,5428,FDP,2021-09-07,19.0
2,5428,CDU/CSU,2021-09-07,19.0
3,5428,AfD,2021-09-07,19.0
4,5428,SPD,2021-09-07,19.0
...,...,...,...,...
4763,615,DIE LINKE,2013-11-18,18.0
4764,615,SPD,2013-11-18,18.0
4765,613,CDU/CSU,2013-10-22,18.0
4766,613,SPD,2013-10-22,18.0


In [38]:
#######################################
#### DESCRIPTIVE STATISTICS OF DATA ###
#######################################

# Count length and number of words of texts
descriptive = df_text_of_parties.copy()
descriptive['text_length'] = descriptive['text'].str.len()
descriptive['num_words'] = descriptive['text'].str.count(' ') + 1
descriptive

In [40]:
# Mean text length per document
mean_text_length = descriptive.groupby(['party', 'text_type']).text_length.mean()
#mean_text_length.to_csv('out/mean_text_length.csv')
mean_text_length

party                  text_type   
AfD                    interruption      3444.901288
                       main_text        57080.665254
BÜNDNIS 90/DIE GRÜNEN  interruption      3874.709205
                       main_text        57242.803758
CDU/CSU                interruption      2676.384134
                       main_text       163445.750520
DIE LINKE              interruption      2269.534737
                       main_text        50638.350731
FDP                    interruption      2408.692308
                       main_text        48171.392405
SPD                    interruption      1813.328358
                       main_text       106301.750520
Name: text_length, dtype: float64

In [41]:
# Mean number of word per document
mean_word_count = descriptive.groupby(['party', 'text_type']).num_words.mean()
#mean_word_count.to_csv('out/mean_word_count.csv')
mean_word_count

party                  text_type   
AfD                    interruption      468.274678
                       main_text        7856.915254
BÜNDNIS 90/DIE GRÜNEN  interruption      520.811715
                       main_text        8079.711900
CDU/CSU                interruption      353.532359
                       main_text       22664.656965
DIE LINKE              interruption      301.105263
                       main_text        7048.592902
FDP                    interruption      321.653846
                       main_text        6787.185654
SPD                    interruption      240.270789
                       main_text       14868.374220
Name: num_words, dtype: float64

In [57]:
# Repeat for only 19th period
descriptive_19 = df_text_of_parties[df_text_of_parties['wahlperiode'] == 19]
descriptive_19['text_length'] = descriptive_19['text'].str.len()
descriptive_19['num_words'] = descriptive_19['text'].str.count(' ') + 1
#with pd.option_context('display.max_rows', None):

descriptive_19

/var/folders/yx/9d3m_ryd2vn67r2brqf_d9cc0000gp/T/ipykernel_8506/1481657341.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  descriptive_19['text_length'] = descriptive_19['text'].str.len()
/var/folders/yx/9d3m_ryd2vn67r2brqf_d9cc0000gp/T/ipykernel_8506/1481657341.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  descriptive_19['num_words'] = descriptive_19['text'].str.count(' ') + 1


,id,party,text,text_type,wahlperiode,datum,text_length,num_words
0,5428,BÜNDNIS 90/DIE GRÜNEN,Sehr geehrter Herr Präsident! Meine Damen und ...,main_text,19.0,2021-09-07,57759,8560
1,5428,FDP,Herr Präsident! Meine Damen und Herren! Der An...,main_text,19.0,2021-09-07,47263,6165
2,5428,CDU/CSU,Sehr geehrter Herr Präsident! Liebe Kolleginne...,main_text,19.0,2021-09-07,63118,9133
3,5428,AfD,Herr Präsident! Meine Damen und Herren! Der Ei...,main_text,19.0,2021-09-07,35106,4877
4,5428,SPD,Herr Präsident! Meine sehr verehrten Damen und...,main_text,19.0,2021-09-07,65231,9449
...,...,...,...,...,...,...,...,...
3804,4702,DIE LINKE,Der ist ja erst später dazugekommen!\nDoch! Is...,interruption,19.0,2017-11-21,2830,376
3805,4702,FDP,"Da sind wir aber jetzt gespannt!\nNein, nein! ...",interruption,19.0,2017-11-21,561,75
3806,4702,CDU/CSU,Was?\nMach ich nie!\nNein!\nJetzt verstehe ich...,interruption,19.0,2017-11-21,2125,266
3807,4702,BÜNDNIS 90/DIE GRÜNEN,Die sind mit gemeint!\nMeine Güte!\nBeifall be...,interruption,19.0,2017-11-21,1579,220


In [73]:
# Total word count per party
mean_word_count_sum_19 = descriptive_19.groupby(['party', 'text_type']).num_words.sum()
#mean_word_count_sum_19.to_csv('out/mean_word_count_sum_19.csv')
mean_word_count_sum_19

party                  text_type   
AfD                    interruption     109108
                       main_text       1854232
BÜNDNIS 90/DIE GRÜNEN  interruption     100234
                       main_text       1644769
CDU/CSU                interruption      84173
                       main_text       5242235
DIE LINKE              interruption      73114
                       main_text       1414376
FDP                    interruption      75267
                       main_text       1603952
SPD                    interruption      66204
                       main_text       3328303
Name: num_words, dtype: int64

In [66]:
# Total text length (num of char) per party
mean_text_length_sum_19 = descriptive_19.groupby(['party', 'text_type']).text_length.sum()
#mean_text_length_sum_19.to_csv('out/mean_text_length_sum_19.csv')
mean_text_length_sum_19

party                  text_type   
AfD                    interruption      802662
                       main_text       13471037
BÜNDNIS 90/DIE GRÜNEN  interruption      741486
                       main_text       11623308
CDU/CSU                interruption      632650
                       main_text       37764347
DIE LINKE              interruption      548327
                       main_text       10161503
FDP                    interruption      563634
                       main_text       11381803
SPD                    interruption      495814
                       main_text       23665643
Name: text_length, dtype: int64

In [68]:
# and the mean...
mean_word_count_19 = descriptive_19.groupby(['party', 'text_type']).num_words.mean()
mean_word_count_19.to_csv('out/mean_word_count_19.csv')
mean_word_count_19

party                  text_type   
AfD                    interruption      468.274678
                       main_text        7856.915254
BÜNDNIS 90/DIE GRÜNEN  interruption      422.928270
                       main_text        6969.360169
CDU/CSU                interruption      355.160338
                       main_text       22119.135021
DIE LINKE              interruption      312.452991
                       main_text        5993.118644
FDP                    interruption      321.653846
                       main_text        6796.406780
SPD                    interruption      281.719149
                       main_text       14043.472574
Name: num_words, dtype: float64

In [72]:
# and the mean...
mean_text_length_19 = descriptive_19.groupby(['party', 'text_type']).text_length.mean()
mean_text_length_19.to_csv('out/mean_text_19_length.csv')
mean_text_length_19

party                  text_type   
AfD                    interruption      3444.901288
                       main_text        57080.665254
BÜNDNIS 90/DIE GRÜNEN  interruption      3128.632911
                       main_text        49251.305085
CDU/CSU                interruption      2669.409283
                       main_text       159343.236287
DIE LINKE              interruption      2343.277778
                       main_text        43057.216102
FDP                    interruption      2408.692308
                       main_text        48227.978814
SPD                    interruption      2109.846809
                       main_text        99855.033755
Name: text_length, dtype: float64